# Panic Project (DHLAB) - Data Preprocessing

author:  `@cyshin971`  

date:    `2025-06-xx`  

version: `1.4`

> version `1.0`: Derived from `data_analysis.ipynb` version `1.0`  
> version `1.4`: as panic null was set to 0 move `panic` remove from `features_dict[dailylog]` before metadata calculation

In [ ]:
version = "1-4"

# 📚 | Import Libraries 

In [ ]:
import config as cfg
import logging

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
logging.getLogger('matplotlib').setLevel(logging.WARNING)

from library.pandas_utils import move_column, remove_columns, aggregate_by_column, create_empty_df, read_csv
from library.text_utils import save_as_csv
from library.json_utils import save_dict_to_file
from library.path_utils import get_file_path

# 📁 | Path Variables 

In [ ]:
DATA_PATH = "./_data"
TMP_PATH = "./cys/_tmp"
OUTPUT_PATH = "./cys/_output"

# ⛏️ | Scraped Data

load preprocessed data (by `junyeol_lee`)
- Each entry are the datapoints for a patient (`ID`) on a specific date (`date`)
- If there were multiple datapoints for a specific date (`date`) for a specific patient (`ID`), the values were processed (`sum`, `avg`, etc.) to a representation for the day
- Questionnaire data was treated as a 'semi-trait' variable
  - if a questionnaire was filled by the patient on a particular day all entries from that point forward will maintain the values of the questionnaire until the patient fills out the questionnaire again
- Diary contents were added (20250613)
	- `mood`, `contents`
- Certain columns were added back (20250613)
  - demography: `suicide_need` (`boolean`)
  - dailylog:
    - `steps_maximum`
	- `steps_mean`
	- `step_hvar_mean`
	- `step_delta`
	- `step_max_delta`
	- `step_mean_delta`
	- `step_hvar_mean_delta`
	- `step_delta2`
	- `step_max_delta2`
	- `step_mean_delta2`
	- `step_hvar_mean_delta2`
	- `steps_variance`

## Scraped Data Features

In [ ]:
scraped_data_filename = "final_result_diary_20250617_03"

features_dict = {
    "scraped_data_filename": scraped_data_filename,
    "preproc_version": version,
	"demography": [
		'gender', 'age', 'marriage', 'job', 'smkHx', 'drinkHx', 'suicideHx', 'suicide_need'
	],
	"dailylog": [
		'panic', 'severity', 'exercise', 'alcohol', 'coffee', 'menstruation',
		'smoking', 'positive_feeling', 'negative_feeling', 'positive_E', 'negative_E',
		'anxiety', 'annoying'
	],
	"lifelog": [
        'HR_var', 'HR_max', 'HR_mean', 'HR_hvar_mean', 'HR_acrophase', 'HR_amplitude', 'HR_mesor',
        'HR_acrophase_difference', 'HR_acrophase_difference_2d', 'HR_amplitude_difference',
        'HR_amplitude_difference_2d', 'HR_mesor_difference', 'HR_mesor_difference_2d',
        'bandpower(0.001-0.0005Hz)', 'bandpower(0.0005-0.0001Hz)', 'bandpower(0.0001-0.00005Hz)', 'bandpower(0.00005-0.00001Hz)',
        'steps', 'SLT1', 'SLT2', 'SLT3', 'SLT4', 'SLT5', 'SLT6', 'total_sleep',
        'steps_maximum', 'steps_mean', 'step_hvar_mean', 'step_delta',
        'step_max_delta', 'step_mean_delta', 'step_hvar_mean_delta',
        'step_delta2', 'step_max_delta2', 'step_mean_delta2', 'step_hvar_mean_delta2', 'steps_variance'
	],
	"questionnaire": [
		'PHQ_9', 'STAI_X2', 'CSM', 'CTQ_1', 'CTQ_2', 'CTQ_3', 'CTQ_4', 'CTQ_5', 'KRQ', 'MDQ',
		'ACQ', 'APPQ_1', 'APPQ_2', 'APPQ_3', 'BSQ', 'GAD_7', 'BRIAN'
	],
	"diary":[
        'mood', 'contents'
	],
	"excluded": [ # Dropped as variables were only in SYM dataset
		'SPAQ_1', 'SPAQ_2', 'BFNE', 'CES_D', 'KOSSSF', 'SADS', 'STAI_X1', 'medication_in_month',
        'Unnamed: 0' # Placeholder column
	],
    "id": [
        'ID', 'date'
    ],
    "label": [
        'panic', 'severity'
    ],
    "metadata": [
        'coffee', 'smoking', 'total_sleep'
    ],
    "metadata_calc": [
        'coffee', 'smoking', 'total_sleep'
    ]
}

demo_vars = features_dict["demography"]
dailylog_vars = features_dict["dailylog"]
lifelog_vars = features_dict["lifelog"]
questionnaire_vars = features_dict["questionnaire"]

state_vars = demo_vars
trait_vars = dailylog_vars + lifelog_vars + questionnaire_vars
all_vars = state_vars + dailylog_vars + lifelog_vars + questionnaire_vars
all_cols = features_dict["id"] + all_vars + features_dict["diary"]

print(f'Number of variables: {len(all_vars)}')
print(f'   Demographic variables: {len(state_vars)}')
print(f'   Daily log variables: {len(dailylog_vars)}')
print(f'   Life log variables: {len(lifelog_vars)}')
print(f'   Questionnaire variables: {len(questionnaire_vars)}')

# _ = save_dict_to_file(features_dict, TMP_PATH, "scraped_features")

## Load Scraped Data

In [ ]:
scraped_data = read_csv(get_file_path(DATA_PATH, scraped_data_filename+'.csv'))

# check if all columns are present
missing_cols = [col for col in all_vars if col not in scraped_data.columns]
if missing_cols:
    logging.warning(f"Missing columns in scraped_data: {missing_cols}")
else:
	logging.info("All expected columns are present in scraped_data.")
extra_cols = [col for col in scraped_data.columns if col not in all_cols + features_dict["excluded"]]
if extra_cols:
	logging.warning(f"Extra columns in scraped_data: {extra_cols}")

# convert date column to datetime format
scraped_data['date'] = pd.to_datetime(scraped_data['date'], format='%Y-%m-%d')
remove_columns(scraped_data, ['Unnamed: 0'])

# remove any of the columns in features_dict["excluded"] if they exist
for col in features_dict["excluded"]:
	if col in scraped_data.columns:
		scraped_data.drop(columns=[col], inplace=True)

print(f"Number of rows: {scraped_data.shape[0]}")
print(f"Number of columns: {scraped_data.shape[1]}")
display(scraped_data.head(5))

# ⚒️ | Data Preprocessing

Changes from scraped data:
- add `entry_id` to identify each entry: `'ID'_'date'`
- add `dataset` to identify source: `SYM1`, `SYM2`, `PXPN`
- convert `panic` (`0`, `1`, `2` = panic) to days befor panic (`dbp`) (panic = `0`, `1`, `2`)
- keep `panic` column instead of removing it (`20250617`)
- add `panic_label` : whether a panic occurred in the entry (`boolean`)
- demographic features were removed from preprocessed data (`data_pre`)
- the data was filtered to remove entries with only demgraphic data

## Initialize Preprocessed Data

- add `entry_id` to identify each entry: `'ID'_'date'`
- add `dataset` to identify source: `SYM1`, `SYM2`, `PXPN`
- convert `panic` (`0`, `1`, `2` = panic) to days befor panic (`dbp`) (panic = `0`, `1`, `2`)
- add `panic_label` (boolean)
- keep `panic` column instead of removing it (`20250617`)
> If using `panic` column as a label this must be removed as a feature from final dataset

In [ ]:
data_pre_init = create_empty_df()
data_pre_init = scraped_data.copy()

# Add 'entry_id' column: unique identifier for each row
data_pre_init['entry_id'] = data_pre_init['ID'] + '_' + data_pre_init['date'].astype(str)
instance_id_unique = data_pre_init['entry_id'].unique()
move_column(data_pre_init, 'entry_id', 0)
print("Number of unique entry IDs:", len(instance_id_unique))
# Check if 'entry_id' is unique
if data_pre_init['entry_id'].duplicated().any():
	# return the rows with duplicate 'entry_id'
	duplicates = data_pre_init[data_pre_init['entry_id'].duplicated(keep=False)]
	display(duplicates.head(5))
	save_as_csv(duplicates, TMP_PATH, f"duplicates_{scraped_data_filename}")
	raise ValueError("Duplicate 'entry_id' found in the data. Please resolve this issue before proceeding.")

# Add 'dataset' column: source of data
data_pre_init['dataset'] = data_pre_init['ID'].str.split('_').str[0]
data_pre_init['dataset'] = data_pre_init['dataset'].str.split('-').str[0]
move_column(data_pre_init, 'dataset', 1)

# Convert 'panic' column to Days Before Panic (dbp)
data_pre_init['dbp'] = data_pre_init.apply(
	lambda row: np.nan if row['panic'] == 0
 				else 0 if row['panic'] == 2 else row['panic'],
	axis=1
)

# Add panic_label column
data_pre_init['panic_label'] = data_pre_init['panic'].apply(lambda x: 1 if x == 2 else 0)

# Update the features_dict
if 'entry_id' not in features_dict['id']:
	features_dict['id'].insert(0, 'entry_id')
if 'dataset' not in features_dict['id']:
	features_dict['id'].append('dataset')
if 'dbp' not in features_dict['dailylog']:
	features_dict['label'].insert(0, 'dbp')
if 'panic_label' not in features_dict['label']:
	features_dict['label'].append('panic_label')
# Remove 'panic' and 'severity' from dailylog features (as they are labels)
if 'panic' in features_dict['dailylog']:
	features_dict['dailylog'].remove('panic')
if 'severity' in features_dict['dailylog']:
	features_dict['dailylog'].remove('severity')

# print scraped_data shape
print(f"Scraped data shape: {scraped_data.shape}")
print(f"Initialized preprocessed data shape: {data_pre_init.shape}")

In [ ]:
display(data_pre_init.head(5))
print("Unique sources in metadata_ljy: ", data_pre_init['dataset'].unique())
print("Number of entries in metadata_ljy:", data_pre_init.shape[0])
sym1_n = data_pre_init[data_pre_init['dataset'] == 'SYM1'].shape[0]
sym2_n = data_pre_init[data_pre_init['dataset'] == 'SYM2'].shape[0]
print("    SYM entries:", sym1_n+sym2_n)
print("    PXPN entries:", data_pre_init[data_pre_init['dataset'] == 'PXPN'].shape[0])
print("Number of unique IDs in metadata_ljy:", len(data_pre_init['ID'].unique()))
# find the unique IDs for SYM1 and SYM2
sym1_ids = data_pre_init[data_pre_init['dataset'] == 'SYM1']['ID'].unique()
sym2_ids = data_pre_init[data_pre_init['dataset'] == 'SYM2']['ID'].unique()
print("    SYM IDs: ", len(sym1_ids)+len(sym2_ids))
print("    PXPN IDs: ", len(data_pre_init[data_pre_init['dataset'] == 'PXPN']['ID'].unique()))
print("Number of panic events (dbp=0):", data_pre_init[data_pre_init['dbp'] == 0].shape[0])

## Initialize Metadata

initialize `metadata` by adding
- `demography_data` : whether demography data exists in the entry (`boolean`)
- `dailylog_data`, `lifelog_data`, `questionnaire_data` : whether each data group exists in the entry (`boolean`)
- `dtype_n` : how many of the 3 `state` groups exists in the entry (`int`)
- `diary_data`: whether panic diary data group exists in the entry (`boolean`)

In [ ]:
metadata_init = create_empty_df()
metadata_init = data_pre_init.copy()

metadata_init['demography_data'] = metadata_init[features_dict['demography']].notnull().any(axis=1).astype(int)
metadata_init['dailylog_data'] = metadata_init[features_dict['dailylog']].notnull().any(axis=1).astype(int)
metadata_init['lifelog_data'] = metadata_init[features_dict['lifelog']].notnull().any(axis=1).astype(int)
metadata_init['questionnaire_data'] = metadata_init[features_dict['questionnaire']].notnull().any(axis=1).astype(int)
metadata_init['diary_data'] = metadata_init[features_dict['diary']].notnull().any(axis=1).astype(int)

# TODO: Diary data is not used in the current analysis, but can be useful for future reference
metadata_init['dtype_n'] = metadata_init['dailylog_data'] + metadata_init['lifelog_data'] + metadata_init['questionnaire_data']
move_column(metadata_init, 'dtype_n', 8)

add_list = ['dailylog_data', 'lifelog_data', 'questionnaire_data', 'dtype_n', 'diary_data']
for item in add_list:
	if item not in features_dict['metadata']:
		features_dict['metadata'].append(item)
del add_list

check_metadata = False
if check_metadata:
    check_type = 'questionnaire' # demography, dailylog, lifelog, questionnaire
    check_for = 0
    test = metadata_init[metadata_init[check_type+'_data'] == check_for].copy()
    test = test[features_dict['id']+features_dict['metadata']+features_dict[check_type]]
    print(f"--------- TEST {test.shape[0]} ENTRIES WITH {check_type} = {check_for} ---------")
    display(test.head(10))
    save_as_csv(test, TMP_PATH, f"metadata_{check_type}_{check_for}")
    print("------------------------------------------------------------------------")
    del test, check_type, check_for

display(metadata_init.head(5))

## Extract Demography Data

- All patients within the scraped data were confirmed to have demographic data (`demography_data` = `True`)
- as such demography_data will not be included in the `metadata`
- Demography data was extracted and saved as `demography.csv` to the `output` directory

In [ ]:
agg_matrix = [
	('gender_n', 'gender', 'nunique'),
	('age_n', 'age', 'nunique'),
	('marriage_n', 'marriage', 'nunique'),
	('job_n', 'job', 'nunique'),
	('smkHx_n', 'smkHx', 'nunique'),
	('drinkHx_n', 'drinkHx', 'nunique'),
	('suicideHx_n', 'suicideHx', 'nunique'),
	('suicide_need_n', 'suicide_need', 'nunique'),
    ('gender', 'gender', 'first'),
	('age', 'age', 'first'),
	('marriage', 'marriage', 'first'),
	('job', 'job', 'first'),
	('smkHx', 'smkHx', 'first'),
	('drinkHx', 'drinkHx', 'first'),
	('suicideHx', 'suicideHx', 'first'),
	('suicide_need', 'suicide_need', 'first'),
]
demo_data = create_empty_df()
demo_data = aggregate_by_column(metadata_init, 'ID', agg_matrix)
# check if the length of each unique value is 1
non_unique_cols = []
for col in features_dict['demography']:
	if demo_data[col+'_n'].apply(lambda x: x > 1).any():
		non_unique_cols.append(col)
if non_unique_cols:
	raise ValueError(f"Demographic columns {non_unique_cols} are not unique for each ID in demo_data.")
else:
	print("All demographic columns are unique for each ID in demo_data.")

for col in features_dict['demography']:
	remove_columns(demo_data, [col+'_n'])
print(f"Number of rows in demo_data: {demo_data.shape[0]}")
display(demo_data.head(5))

save_as_csv(demo_data, OUTPUT_PATH, f"panic_demography_data_{version}({scraped_data_filename})")

## Extract Panic Diary Data

In [ ]:
if all(col in data_pre_init.columns for col in features_dict['diary']):
	panic_diary_data = create_empty_df()
	panic_diary_data = data_pre_init[features_dict['id'] + features_dict['diary']].copy()

	panic_diary_entries = metadata_init[metadata_init['diary_data'] == 1]['entry_id'].unique()
	# Filter panic_diary_data to only include entries with diary data
	panic_diary_data = panic_diary_data[panic_diary_data['entry_id'].isin(panic_diary_entries)]

	print(f"Number of rows in panic_diary_data: {panic_diary_data.shape[0]}")
	print(f"Number of unique patients in panic_diary_data: {panic_diary_data['ID'].nunique()}")
	print(f"Unique datasets in panic_diary_data: {panic_diary_data['dataset'].unique()}")
	display(panic_diary_data.head(5))

	save_as_csv(panic_diary_data, OUTPUT_PATH, f"panic_diary_data_{version}({scraped_data_filename})")
	remove_columns(data_pre_init, features_dict['diary'])  # Remove diary columns from data_pre_init
else:
	print("No diary data found in the scraped data. Skipping panic_diary_data creation.")

## Construct Intermediate Metadata
- the current `metadata` (`metadata_init`) was filtered to include only columns for identification, added columns for metadata, and labels
- the `metadata` was also filtered to get rid of all entries that only have demography data (`dtype_n` = 0)

In [ ]:
metadata_int = create_empty_df()
metadata_int = metadata_init.copy()

metadata_int = metadata_int[features_dict['id'] + features_dict['metadata'] + features_dict['label']]
move_column(metadata_int, 'severity', -1)
move_column(metadata_int, 'panic_label', -1)
# metadata_int = metadata_int[metadata_int['dtype_n'] > 0]
metadata_int = metadata_int[metadata_int['date'].notnull()]
display(metadata_int.head(5))

## Filter Preprocessed Data

- demographic features were removed from preprocessed data (`data_pre`)
- the data was filtered to remove entries with only demgraphic data
- the removed IDs were checked to see if no relevant entries were discarded

In [ ]:
data_pre = create_empty_df()
data_pre = data_pre_init.copy()
# Remove demographic features from data_proc
remove_columns(data_pre, features_dict['demography'])
# Filter data_proc to keep only rows with entry IDs present in metadata_int
metadata_int_unique_ids = metadata_int['entry_id'].unique()
data_pre = data_pre[data_pre['entry_id'].isin(metadata_int_unique_ids)]

# remove rows with null dates
data_pre = data_pre[data_pre['date'].notnull()]

# Find IDs present in unfiltered_data but missing in filtered_data (i.e., lost after filtering)
check_missing_ids = False
if check_missing_ids:
	missing_ids = np.setdiff1d(data_pre_init['ID'].unique(), data_pre['ID'].unique())
	missing_data = data_pre_init[data_pre_init['ID'].isin(missing_ids)]
	print(f"Number of IDs lost after filtering: {len(missing_ids)}")
	_ = save_as_csv(missing_data, TMP_PATH, f"missing_{scraped_data_filename}")

## 💾 | Save Preprocessed Data

In [ ]:
# save data_pre to CSV
save_as_csv(data_pre, OUTPUT_PATH, f"panic_pre_data_{version}({scraped_data_filename})")

display(data_pre.head(3))
print("--------------------------------------------------------")
print("Total entries in original: ", data_pre_init.shape[0])
sym1_n = data_pre_init[data_pre_init['dataset'] == 'SYM1'].shape[0]
sym2_n = data_pre_init[data_pre_init['dataset'] == 'SYM2'].shape[0]
print("    SYM entries:", sym1_n+sym2_n)
print("    PXPN entries:", data_pre_init[data_pre_init['dataset'] == 'PXPN'].shape[0])
print("Number of unique IDs in original:", len(data_pre_init['ID'].unique()))
# find the unique IDs for SYM1 and SYM2
sym1_ids = data_pre_init[data_pre_init['dataset'] == 'SYM1']['ID'].unique()
sym2_ids = data_pre_init[data_pre_init['dataset'] == 'SYM2']['ID'].unique()
print("    SYM IDs: ", len(sym1_ids)+len(sym2_ids))
print("    PXPN IDs: ", len(data_pre_init[data_pre_init['dataset'] == 'PXPN']['ID'].unique()))
print("Number of panic events (dbp=0):", data_pre_init[data_pre_init['dbp'] == 0].shape[0])
print("--------------------------------------------------------")
print("Total entries in filtered: ", data_pre.shape[0])
sym1_n = data_pre[data_pre['dataset'] == 'SYM1'].shape[0]
sym2_n = data_pre[data_pre['dataset'] == 'SYM2'].shape[0]
print("    SYM entries:", sym1_n+sym2_n)
print("    PXPN entries:", data_pre[data_pre['dataset'] == 'PXPN'].shape[0])
print("Number of unique IDs in filtered:", len(data_pre['ID'].unique()))
# find the unique IDs for SYM1 and SYM2
sym1_ids = data_pre[data_pre['dataset'] == 'SYM1']['ID'].unique()
sym2_ids = data_pre[data_pre['dataset'] == 'SYM2']['ID'].unique()
print("    SYM IDs: ", len(sym1_ids)+len(sym2_ids))
print("    PXPN IDs: ", len(data_pre[data_pre['dataset'] == 'PXPN']['ID'].unique()))
print("Number of panic events (dbp=0):", data_pre[data_pre['dbp'] == 0].shape[0])

# 📖 | Metadata

**Description**
- `entry_id`: ID for each entry `'ID'_'date'`
- `ID`: ID for each patient
- `date`: logging date of each entry
- `dataset`: source of entry (`SYM1`, `SYM2`, `PXPN`)
- `dailylog_data`: whether daily log data exists in the entry (`boolean`)
- `lifelog_data`: whether life log data exists in the entry (`boolean`)
- `questionnaire_data`: whether questionnaire data exists in the entry (`boolean`)
- `dtype_n`: how many of the 3 `state` groups exists in the entry (`int`)
- `diary_data`: whether panic diary data exists in the entry (`boolean`)
- `dbp`: number of consecutive days prior to panic. i.e. panic day = 0; 1 day prior = 1; etc. (up to 3)
- `n_prior_data`: number of existing consecutive prior (days) entries
- `ref_event_id`: the `entry_id` to which days before panic (`dbp`) is referencing
- `valid_entry_3`: whether the entry has 3 consecutive days of prior data (`n_prior_data`)
- `valid_entry_2`: whether the entry has 2 consecutive days of prior data (`n_prior_data`)
- `valid_entry_1`: whether the entry has 1 consecutive days of prior data (`n_prior_data`)
- `panic_label`: whether a panic occured in the entry (`boolean`)
- `severity`: severity of the panic (1 ~ 5)

## Calculate Days Before Panic (``dbp``) and Prior Consecutive Days (``n_prior_data``)

- calculate the consecutive 'days before panic' (`dbp`):
  - day when panic occured -> `dbp` = 0
  - 1 day before panic -> `dbp` = 1
  - 2 day before panic -> `dbp` = 2
  - 3 day before panic -> `dbp` = 3 (etc)
  - stop calculating at a set limit (`delta_days`) or if a panic occurred within the limit
- calculate the number of existing prior consecutive (days) entries (`n_prior_data`) (Default: 3)
  - stop calculating at a certain limit (`lookback_limit`) (Default: 7)

> May take ~ 1 to 2 min

In [ ]:
metadata_calc = create_empty_df()
metadata_calc = metadata_int.copy()

metadata_calc['n_prior_data']    = None
metadata_calc['ref_event_id']    = None
move_column(metadata_calc, 'panic_label', -1)
move_column(metadata_calc, 'severity', -1)
metadata_calc.sort_values(by=['ID', 'date'], ascending=False, inplace=True)

d_days = 3
l_back_lim = 7

def calculate_days_before_panic(df, patient_id, delta_days=3, lookback_limit=7):
    patient_data = df[df['ID'] == patient_id]
    entry_dates_series = patient_data['date']
    if len(set(entry_dates_series)) != len(entry_dates_series):
        raise ValueError(f"Duplicate dates found for patient {patient_id}. Please check the data.")
    panic_dates_series = patient_data[patient_data['dbp'] == 0]['date']
    if len(set(panic_dates_series)) != len(panic_dates_series):
        raise ValueError(f"Duplicate panic dates found for patient {patient_id}. Please check the data.")
    
    entry_dates = set(entry_dates_series)
    panic_dates = set(panic_dates_series)

    for panic_date in sorted(panic_dates, reverse=True): # Sort from latest to earliest
        index = patient_data[patient_data['date'] == panic_date].index[0]
        event_id = patient_data.loc[index, 'entry_id']
        df.loc[index, 'dbp'] = 0
        for j in range(1, delta_days + 1):
            prior_date = panic_date - pd.Timedelta(days=j)
            if prior_date in entry_dates:
                index = patient_data[patient_data['date'] == prior_date].index[0]
                df.loc[index, 'dbp'] = j
                df.loc[index, 'ref_event_id'] = event_id
    
    for entry_date in sorted(entry_dates, reverse=True):
        for j in range(1, lookback_limit + 1):
            if j == lookback_limit+1:
                df.loc[index, 'n_prior_data'] = j
                break
            prior_date = entry_date - pd.Timedelta(days=j)
            if prior_date not in entry_dates:
                break
            index = patient_data[patient_data['date'] == prior_date].index[0]
            if df.loc[index, 'panic_label'] == 1:
                break
            index = patient_data[patient_data['date'] == entry_date].index[0]
            df.loc[index, 'n_prior_data'] = j

def process_calculate_days_before_panic(df, delta_days=3, lookback_limit=7):
    patient_ids = df['ID'].unique()
    for patient_id in patient_ids:
        calculate_days_before_panic(df, patient_id, delta_days, lookback_limit)
        progress = (np.where(patient_ids == patient_id)[0][0] + 1) / len(patient_ids) * 100
        print(f"Processing: {progress:.2f}% complete", end='\r')
    # replace None values in 'n_prior_data' with 0
    df['n_prior_data'] = df['n_prior_data'].fillna(0).astype(int)
    return df

metadata_int = process_calculate_days_before_panic(metadata_calc, delta_days=d_days, lookback_limit=l_back_lim)

# update features_dict with metadata columns
if 'ref_event_id' not in features_dict['metadata']:
	features_dict['metadata'].append('ref_event_id')
if 'n_prior_data' not in features_dict['metadata']:
	features_dict['metadata'].append('n_prior_data')

In [ ]:
p_id = 'SYM2-1-422'
disp_df = metadata_int[metadata_int['ID'] == p_id]
display(disp_df.head(10))
del disp_df, p_id

## Find Valid Entries
- add `valid_entry_3`: whether the entry has 3 consecutive days of prior data (`n_prior_data`)
- add `valid_entry_2`: whether the entry has 2 consecutive days of prior data (`n_prior_data`)
- add `valid_entry_1`: whether the entry has 1 consecutive days of prior data (`n_prior_data`)

In [ ]:
metadata_int['valid_entry_3'] = metadata_int.apply(
	lambda row: 1 if row['n_prior_data'] >= 3 else 0,
	axis=1
)
metadata_int['valid_entry_2'] = metadata_int.apply(
	lambda row: 1 if row['n_prior_data'] >= 2 else 0,
	axis=1
)
metadata_int['valid_entry_1'] = metadata_int.apply(
	lambda row: 1 if row['n_prior_data'] >= 1 else 0,
	axis=1
)
move_column(metadata_int, 'ref_event_id', -1)
move_column(metadata_int, 'panic_label', -1)
move_column(metadata_int, 'severity', -1)
display(metadata_int.head(5))

In [ ]:
test_panic_dbpnot0 = metadata_int[(metadata_int['panic'] == 2) & (metadata_int['dbp'] != 0)]['entry_id'].unique()
test_panic_dbp1 = metadata_int[(metadata_int['panic'] == 1) & (metadata_int['dbp'] != 1)]['entry_id'].unique()
if len(test_panic_dbpnot0) != 0:
	raise ValueError("Entries found with dbp != 0 for panic events. Please check the data.")
if len(test_panic_dbp1) != 0:
	raise ValueError("Entries found with dbp != 1 for panic = 1. Please check the data.")
del test_panic_dbpnot0, test_panic_dbp1

## 💾 | Save Metadata

In [ ]:
metadata = create_empty_df()
metadata = metadata_int.copy()
save_as_csv(metadata, OUTPUT_PATH, f"panic_metadata_{version}({scraped_data_filename})")
save_dict_to_file(features_dict, OUTPUT_PATH, "panic_features_dict")

display(metadata.head(10))

# 🔍 | Data Analysis

In [ ]:
print("Scraped data shape:", scraped_data.shape)
display(scraped_data.head(2))
print("Data preprocessed shape:", data_pre.shape)
display(data_pre.head(2))
print("Metadata shape:", metadata.shape)
display(metadata.head(2))

In [ ]:
scraped_data_n = len(scraped_data)
data_pre_entry_ids = data_pre['entry_id'].unique()
print(f"Scraped Entries: {scraped_data_n} -> {len(data_pre_entry_ids)} after preprocessing. discarded {scraped_data_n - len(data_pre_entry_ids)} entries.")

In [ ]:
scraped_data_ids = scraped_data['ID'].unique()
data_pre_ids = data_pre['ID'].unique()
print(f"Scraped Data (n): {len(scraped_data_ids)} -> Preprocessed Data (n): {len(data_pre_ids)}")
print(f"{len(scraped_data_ids) - len(data_pre_ids)} patient data were discarded during preprocessing due to entries having only demographic data")

In [ ]:
metadata_panic = metadata[metadata['panic_label'] == 1].copy()
print(f"Number of panic events in preprocessed data: {metadata_panic.shape[0]}")
agg_matrix = [
	('valid_entries_3', 'valid_entry_3', 'sum'),
	('valid_entries_2', 'valid_entry_2', 'sum'),
	('valid_entries_1', 'valid_entry_1', 'sum'),
]
agg_metadata_panic = aggregate_by_column(metadata_panic, 'ID', agg_matrix)

print(f"Number of valid panic entries (valid_entry_3): {agg_metadata_panic['valid_entries_3'].sum()}")
print(f"Number of valid panic entries (valid_entry_2): {agg_metadata_panic['valid_entries_2'].sum()}")
print(f"Number of valid panic entries (valid_entry_1): {agg_metadata_panic['valid_entries_1'].sum()}")
print("--------------------------------------------------------")
print(f"Number of patients with valid panic entries (valid_entry_3): {agg_metadata_panic[agg_metadata_panic['valid_entries_3'] > 0].shape[0]}")
print(f"Number of patients with valid panic entries (valid_entry_2): {agg_metadata_panic[agg_metadata_panic['valid_entries_2'] > 0].shape[0]}")
print(f"Number of patients with valid panic entries (valid_entry_1): {agg_metadata_panic[agg_metadata_panic['valid_entries_1'] > 0].shape[0]}")